# Mixfeat session-level experiments

In [1]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from typing import Tuple
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from pathlib import Path
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
from aif360.algorithms.postprocessing import EqOddsPostprocessing
from aif360.datasets import BinaryLabelDataset

# update the path so we can directly import code from the DVlog
sys.path.append(os.path.dirname(os.path.abspath(os.path.join(os.getcwd(), os.pardir, "DVlog"))))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir, "DVlog")))

from DVlog.utils.bias_mitigations import apply_oversampling, apply_mixfeat_oversampling
from DVlog.utils.metrics import calculate_performance_measures, calculate_gender_performance_measures, calculate_fairness_measures

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[inFairness]'


In [2]:
annotations_file = Path(r"../DVlog/dataset/dvlog_labels_v2.csv")
embeddings_path = Path("../DVlog/dataset/sent-embeddings-dataset")
feature_name = "sent_mpnet_keyw"

random_seeds = [0, 1, 42, 1123, 3407]

In [3]:
# load in the annotation labels
df_annotations = pd.read_csv(annotations_file)
df_annotations.reset_index(drop=True, inplace=True)
df_annotations.head()

,video_id,label,gender,dataset
0,0,1,f,train
1,1,1,f,test
2,2,1,m,train
3,3,1,m,train
4,4,1,f,test


In [4]:
# loop over each row and compute the average embeddings
df_annotations["avg_embed"] = None

# loop over each row and retrieve the embeddings
seq_length = 104

for idx, row in df_annotations.iterrows():
    # get the texts
    video_id = row.video_id
    
    # setup the path to the file
    embedding_path = os.path.join(embeddings_path, str(video_id), f"{feature_name}.npy")
    embedding = np.load(embedding_path).astype(np.float32)

    # apply the padding
    padded_embedding = embedding[:seq_length]

    # get the average over the whole embedding
    avg_embedding = np.mean(padded_embedding, axis=0)
    # std_embedding = np.std(padded_embedding, axis=0)

    # put the embedding back
    df_annotations.at[idx, "avg_embed"] = avg_embedding

df_annotations.head()

,video_id,label,gender,dataset,avg_embed
0,0,1,f,train,"[-0.0043204557, 0.0025047027, -0.022133984, -0..."
1,1,1,f,test,"[0.014702894, 0.017551864, -0.01323786, -0.016..."
2,2,1,m,train,"[-0.0020621587, -0.002233186, -0.009010282, -0..."
3,3,1,m,train,"[0.013287175, 0.005526411, -0.010409681, -0.02..."
4,4,1,f,test,"[-0.008224284, 0.02129893, -0.0096479375, -0.0..."


In [5]:
# setup the train and validation datasets
train_indices = df_annotations[df_annotations["dataset"] == "train"].index
val_indices = df_annotations[df_annotations["dataset"] == "val"].index

# prepare the features and labels
avg_features = np.stack(df_annotations["avg_embed"].values)
labels = df_annotations["label"].values
genders = df_annotations["gender"].values

# create the train and validation sets
X_train, X_val = avg_features[train_indices], avg_features[val_indices]
y_train, y_val = labels[train_indices], labels[val_indices]

# combine the train and validation sets
X = np.vstack((X_train, X_val))
y = np.hstack((y_train, y_val))

# Create a test_fold array: -1 for training set, 0 for validation set
test_fold = np.concatenate([
    -1 * np.ones(len(X_train), dtype=int),
    np.zeros(len(X_val), dtype=int)
])

print(X.shape, y.shape, test_fold.shape)

# Create PredefinedSplit object
ps = PredefinedSplit(test_fold)

(662, 768) (662,) (662,)


## setup the gridsearch with the parameters
- C (Regularization Parameter): Controls the trade-off between achieving a low error on the training data and minimizing the norm of the weights. A small value for C makes the decision surface smooth, while a large value of C aims to classify all training examples correctly.

- Gamma (Kernel Coefficient): Defines how far the influence of a single training example reaches, with low values meaning 'far' and high values meaning 'close'. It is applicable for 'rbf', 'poly', and 'sigmoid' kernels.

- Kernel: Specifies the kernel type to be used in the algorithm. Common kernels are 'linear', 'poly' (polynomial), 'rbf' (radial basis function), and 'sigmoid'.

In [6]:
# Define the SVM and parameter grid
svm = SVC(random_state=42)
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}

# Set up and run GridSearchCV
grid_search_avg = GridSearchCV(estimator=svm, param_grid=param_grid, cv=ps, verbose=2, n_jobs=-1)
grid_search_avg.fit(X, y)

Fitting 1 folds for each of 64 candidates, totalling 64 fits


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=SVC(random_state=42), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             verbose=2)

In [7]:
# Output best parameters and score
best_params = grid_search_avg.best_params_
print("Best parameters found: ", best_params)
print("Best validation score: ", grid_search_avg.best_score_)

Best parameters found:  {'C': 10, 'gamma': 1, 'kernel': 'poly'}
Best validation score:  0.9523809523809523


In [8]:
# build the function for automatically retrieve all metrics
def evaluate_model(y_true, y_pred, protected):

    # calculate the performance metrics
    w_precision, w_recall, w_fscore, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")

    # calculate the fairness metrics
    eq_oppor, eq_acc, pred_equal, _, _ = calculate_fairness_measures(y_true, y_pred, protected, unprivileged='m')
    
    # eq_oppor, eq_acc, fairl_eq_odds, unpriv_stats, priv_stats = calculate_fairness_measures(y_true, y_pred, protected, 'm')
    gender_metrics = calculate_gender_performance_measures(y_true, y_pred, protected)

    measure_dict = {
        "precision": w_precision,
        "recall": w_recall,
        "fscore": w_fscore,
        f"{gender_metrics[0][0]}_fscore": gender_metrics[0][3],
        f"{gender_metrics[1][0]}_fscore": gender_metrics[1][3],
        "eq_oppor": eq_oppor,
        "eq_acc": eq_acc,
        "pred_eq": pred_equal}
    return measure_dict

In [9]:
# evaulate this model on the test set
test_indices = df_annotations[df_annotations["dataset"] == "test"].index
X_test, y_test, protec_test = avg_features[test_indices], labels[test_indices], genders[test_indices]

# Evaluate the best model (avg)
best_svm = grid_search_avg.best_estimator_
y_pred = best_svm.predict(X_test)
base_eval_dict_avg = evaluate_model(y_test, y_pred, protec_test)

# Evaluate the best model (std)
# best_svm = grid_search_std.best_estimator_
# y_pred = best_svm.predict(X_test_std)
# base_eval_dict_std = evaluate_model(y_test, y_pred, protec_test)

## Setup the bias mitigations

In [10]:
mixfeat_options = ['oversample', 'group_upsample', 'mixgender_upsample', 'subgroup_upsample', 'synthetic', 'synthetic_mixgendered']
results = [("base_model_avg", base_eval_dict_avg)]

# get the training section
df_train = df_annotations[df_annotations["dataset"] == "train"]

# take the training_df and do the oversampling for each option
for seed in random_seeds:
    for option in mixfeat_options:
        print(f"Processing: {option} with seed: {seed}")

        # get the training section
        df_copy = df_train.copy()
        if option == 'oversample':
            training_df = apply_oversampling(df_copy, seed)
            X = np.stack(training_df["avg_embed"].values)

        else:
            training_df = apply_mixfeat_oversampling(df_copy, option, 1, seed)

            # extract the training data and apply the mixfeat operation whenever possible
            X = []
            for _, row in training_df.iterrows():
                if row.mixfeat:
                    idx1, idx2 = row.mixfeat
                    prob = row.mixfeat_probs[0]

                    # get the embeddings from the dataframe
                    embedding1 = df_train.loc[df_train['video_id'] == idx1]["avg_embed"].values[0]
                    embedding2 = df_train.loc[df_train['video_id'] == idx2]["avg_embed"].values[0]

                    final_embedding = (embedding1 * prob) + (embedding2 * (1 - prob))
                    X.append(final_embedding)
                else:
                    X.append(row.avg_embed)

            # get all the information and train the model
            X = np.array(X)

        # retrieve the label information
        y = training_df["label"].values

        # train an SVM model
        svm = SVC(**best_params, random_state=seed)
        svm.fit(X, y)

        # evaluate the model
        y_pred = svm.predict(X_test)
        eval_dict = evaluate_model(y_test, y_pred, protec_test)
        results.append((option, eval_dict))

Processing: oversample with seed: 0
Processing: group_upsample with seed: 0
Processing: mixgender_upsample with seed: 0
Processing: subgroup_upsample with seed: 0
Processing: synthetic with seed: 0
Processing: synthetic_mixgendered with seed: 0
Processing: oversample with seed: 1
Processing: group_upsample with seed: 1
Processing: mixgender_upsample with seed: 1
Processing: subgroup_upsample with seed: 1
Processing: synthetic with seed: 1
Processing: synthetic_mixgendered with seed: 1
Processing: oversample with seed: 42
Processing: group_upsample with seed: 42
Processing: mixgender_upsample with seed: 42
Processing: subgroup_upsample with seed: 42
Processing: synthetic with seed: 42
Processing: synthetic_mixgendered with seed: 42
Processing: oversample with seed: 1123
Processing: group_upsample with seed: 1123
Processing: mixgender_upsample with seed: 1123
Processing: subgroup_upsample with seed: 1123
Processing: synthetic with seed: 1123
Processing: synthetic_mixgendered with seed: 1

## Setup the post-processing bias mitigation

In [11]:
# get the results from the normal textual model
best_svm = grid_search_avg.best_estimator_
y_pred_train = best_svm.predict(X_train)
y_pred_test = best_svm.predict(X_test)

# set 
protec_train = genders[train_indices]
protected_train = np.where(protec_train == "m", 0, 1)
protected_test = np.where(protec_test == "m", 0, 1)
    
# Function to apply EqOddsPostprocessing
def apply_eqodds(y_train_true, y_train_pred, y_test_pred, protected_attr_train, protected_attr_test, seed):
    # Create BinaryLabelDataset for training data
    dataset_train_true = BinaryLabelDataset(favorable_label=1, unfavorable_label=0, df=pd.DataFrame({
        'label': y_train_true,
        'protected': protected_attr_train
    }), label_names=['label'], protected_attribute_names=['protected'])

    dataset_train_pred = BinaryLabelDataset(favorable_label=1, unfavorable_label=0, df=pd.DataFrame({
        'label': y_train_pred,
        'protected': protected_attr_train
    }), label_names=['label'], protected_attribute_names=['protected'])

    # Create BinaryLabelDataset for test data
    dataset_test_pred = BinaryLabelDataset(favorable_label=1, unfavorable_label=0, df=pd.DataFrame({
        'label': y_test_pred,
        'protected': protected_attr_test
    }), label_names=['label'], protected_attribute_names=['protected'])

    # Apply EqOddsPostprocessing
    eq_odds = EqOddsPostprocessing(unprivileged_groups=[{'protected': 0}],
                                   privileged_groups=[{'protected': 1}], seed=seed)

    eq_odds = eq_odds.fit(dataset_train_true, dataset_train_pred)
    dataset_transf_test_pred = eq_odds.predict(dataset_test_pred)

    # Extract the adjusted predictions
    adjusted_pred = dataset_transf_test_pred.labels.ravel()
    return adjusted_pred

# 
for seed in random_seeds:
    new_preds = apply_eqodds(y_train, y_pred_train, y_pred_test, protected_train, protected_test, seed)
    eval_dict = evaluate_model(y_test, new_preds, protec_test)
    results.append(("eqodds", eval_dict))

In [12]:
# Extract data into a structured format
extracted_data = []
for name, result in results:
    data = {
        "name": name,
        "Precision": np.round(result["precision"], 3),
        "Recall": np.round(result["recall"], 3),
        "F-score": np.round(result["fscore"], 3),
        "Male F-score": np.round(result["m_fscore"], 3),
        "Female F-score": np.round(result["f_fscore"], 3),
        "eq_oppor": np.round(result["eq_oppor"], 2),
        "eq_acc": np.round(result["eq_acc"], 2),
        "pred_eq": np.round(result["pred_eq"], 2)
    }
    extracted_data.append(data)

# Convert the list of dictionaries to a pandas DataFrame and display it
df = pd.DataFrame(extracted_data)
df.groupby("name").mean()

,Precision,Recall,F-score,Male F-score,Female F-score,eq_oppor,eq_acc,pred_eq
name,,,,,,,,
base_model_avg,0.9220,0.9210,0.9210,0.8950,0.9350,0.880,0.960,0.720
eqodds,0.9160,0.9150,0.9150,0.8770,0.9350,0.880,0.940,0.970
group_upsample,0.9164,0.9162,0.9162,0.8804,0.9350,0.856,0.944,0.722
mixgender_upsample,0.9140,0.9138,0.9138,0.8760,0.9332,0.818,0.942,0.496
oversample,0.9190,0.9174,0.9174,0.8876,0.9330,0.902,0.954,0.938
subgroup_upsample,0.9042,0.8886,0.8864,0.8538,0.9040,0.976,0.948,1.268
synthetic,0.9216,0.9198,0.9198,0.8948,0.9330,0.916,0.962,0.936
synthetic_mixgendered,0.8708,0.8300,0.8228,0.7990,0.8352,1.000,0.958,1.130


In [13]:
df.groupby("name").std()

,Precision,Recall,F-score,Male F-score,Female F-score,eq_oppor,eq_acc,pred_eq
name,,,,,,,,
base_model_avg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
eqodds,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
group_upsample,0.002608,0.002683,0.002683,0.007603,0.000000,0.013416,0.008944,0.173263
mixgender_upsample,0.002828,0.002683,0.002683,0.000000,0.004025,0.010954,0.004472,0.035777
oversample,0.005788,0.005367,0.005367,0.015710,0.004472,0.045497,0.019494,0.127945
subgroup_upsample,0.001789,0.003286,0.003578,0.008319,0.004472,0.013416,0.010954,0.064962
synthetic,0.006580,0.006573,0.006573,0.012377,0.004472,0.035071,0.010954,0.215012
synthetic_mixgendered,0.002490,0.004243,0.004604,0.000000,0.007430,0.000000,0.010954,0.042426
